In [1]:
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from dateutil.rrule import rrule, MONTHLY
from pprint import pprint as pp
import json
import os
import shutil
import numpy as np
from collections import defaultdict

In [2]:
from modules.aux_functions import *
from modules.global_vars import *
from modules.reservation_management import get_reservation_code
from modules.table_manager import manage_table, write_table_data, get_table_data

In [3]:
np.random.seed(583)

In [4]:
def manage_hotel_table(reservation, hotel_table_path):
    """FIXME! briefly describe function

    :param reservation: 
    :param hotel_table_path: 
    :returns: 
    :rtype: 

    """
    table_code = get_reservation_code(reservation)

    file_path = hotel_table_path + table_code + TABLES_EXTENSION

    table_data = get_table_data(file_path)

    # Generate competititon price code
    our_price = str(round_price(reservation[PRICE], UNITS))

    # Case of our_price as key
    # Retrieve value of the table and set it to 0 if not exists
    
    temp_date = reservation[TIMESTAMP].strftime("%d/%m/%Y")
    if our_price in table_data.keys():
        table_data[our_price][RESERVATIONS] += 1
        temp_dates = set(table_data[our_price][RES_DAYS])
        temp_dates.add(temp_date)
        table_data[our_price][RES_DAYS] = list(temp_dates)
    else:
        table_data[our_price] = {}
        table_data[our_price][RESERVATIONS] = 1
        table_data[our_price][RES_DAYS] = [temp_date]

    write_table_data(table_data, file_path)


In [5]:
hotel_days = {}

for hotel in ALL_HOTELS:

    hotel_table_path = TEMP_HOTEL_TABLES_PATH + hotel + "/"

    hotel_path = "Dataset/Hoteles/obatch- " + hotel
    # Load data
    df = pd.read_csv(hotel_path)
    df = cleanData(df)

    # pp(df[ROOM].unique())

    if os.path.exists(hotel_table_path):
        shutil.rmtree(hotel_table_path)
    os.mkdir(hotel_table_path)
    print(hotel_table_path)
    
    hotel_days[hotel] = (df[TIMESTAMP].max() - df[TIMESTAMP].min()).days

    res_per_month = defaultdict(int)
    
    #for dt in rrule(MONTHLY, dtstart=df[TIMESTAMP].min(), until=df[TIMESTAMP].max()):
    #    res_per_month[dt.strftime("%m/%Y")] = 0
            
    for index, reservation in df.iterrows():
        res_per_month[reservation[TIMESTAMP].strftime("%m/%Y")] += 1
        if get_reservation_code(reservation) == "ERROR":
            continue
        if reservation[DAYS_IN_ADVANCE] <= 31:
            manage_hotel_table(reservation, hotel_table_path)
    
    pp(res_per_month)

Temp_tables/Melia Barajas/
defaultdict(<class 'int'>,
            {'01/2018': 30960,
             '02/2018': 630035,
             '03/2018': 20874})
Temp_tables/Axor Barajas/
defaultdict(<class 'int'>,
            {'01/2018': 367517,
             '02/2018': 327209,
             '03/2018': 9394,
             '09/2017': 24,
             '10/2017': 223237,
             '11/2017': 255907,
             '12/2017': 45872})
Temp_tables/Senator Barajas/
defaultdict(<class 'int'>,
            {'01/2018': 387313,
             '02/2018': 338988,
             '03/2018': 10549,
             '09/2017': 24,
             '10/2017': 297039,
             '11/2017': 340635,
             '12/2017': 49645})


In [6]:
hotel_days

{'Melia Barajas': 56, 'Axor Barajas': 154, 'Senator Barajas': 154}

In [7]:
# Proceso para calcula el número medio de clientes por día, distribuyendo los días totales de reservas
# de manera proporcional entre los precios, y utilizando como proporción el núero de días distintos que
# se ha puesto cada precio.

for hotel in ALL_HOTELS:
    final_table_path = HOTEL_TABLES_PATH + hotel + "/"
    hotel_table_path = TEMP_HOTEL_TABLES_PATH + hotel + "/"
    hotel_price_days_path = PRICE_DAYS_TABLES_PATH + hotel + "/"
    
    if os.path.exists(final_table_path):
        shutil.rmtree(final_table_path)
    os.makedirs(final_table_path)
    print(final_table_path)
    
    if os.path.exists(hotel_price_days_path):
        shutil.rmtree(hotel_price_days_path)
    os.makedirs(hotel_price_days_path)
    
    for file_name in os.listdir(hotel_table_path):
        table_file = hotel_table_path + file_name
        final_file = final_table_path + file_name
        final_price_days_file = hotel_price_days_path + file_name
        
        with open(table_file) as fp:
            hotel_data = json.load(fp)
        
        total_days = 0
        for price in hotel_data:
            total_days += len(hotel_data[price][RES_DAYS])
        
        price_days_dict = {}
        out_data = {}
        for price in hotel_data:
            out_data[price] = hotel_data[price][RESERVATIONS] / (
                hotel_days[hotel] * (len(hotel_data[price][RES_DAYS]) / total_days))
            price_days_dict[price] = hotel_days[hotel] * (len(hotel_data[price][RES_DAYS]) / total_days)
        
        write_table_data(out_data, final_file)
        write_table_data(price_days_dict, final_price_days_file)

Tablas_hotel/Melia Barajas/
Tablas_hotel/Axor Barajas/
Tablas_hotel/Senator Barajas/
